In [3]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import scipy

matplotlib.rcParams['figure.figsize'] = [15, 15]

In [4]:
import time
import os
import datetime

In [5]:
import tensorflow as tf
import keras

from keras import regularizers, optimizers
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, Dense, Activation, Flatten, Reshape
from keras.layers import Dropout, BatchNormalization
from keras.layers import Softmax, ReLU, LeakyReLU, Lambda 
from keras.layers import Conv2D, MaxPooling2D, SeparableConv2D
from keras.layers import LSTM, GRU
from keras.layers import concatenate

from keras.callbacks import TensorBoard, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' 

In [6]:
# shape of model is 
# input: 26 (A-Z), 1 (Value of letter scaled between 1-99), 5 (None, 2L, 3L, 2W, 3W)
# timestep: 16 (max word length)
# output: score normalised to 0-1000
# type of activation: ReLU
def create_lstm_model(timesteps, input_dim):
    alpha = 0.2

    inputs = Input(shape=(timesteps, input_dim))

    x = inputs
    x = LSTM(256)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(64)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(32)(x)
    x = LeakyReLU(alpha)(x)

    x = Dense(16)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(1)(x)
    x = LeakyReLU(alpha)(x)
    
    outputs = x

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [7]:
# shape of model is 
# input: 26 (A-Z), 1 (Value of letter scaled between 1-99), 5 (None, 2L, 3L, 2W, 3W)
# timestep: 16 (max word length)
# output: score normalised to 0-1000
# type of activation: ReLU
def create_gru_model(timesteps, input_dim):
    alpha = 0.2

    inputs = Input(shape=(timesteps, input_dim))

    x = inputs
    x = GRU(256)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(64)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(32)(x)
    x = LeakyReLU(alpha)(x)

    x = Dense(16)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(1)(x)
    x = LeakyReLU(alpha)(x)
    
    outputs = x

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [8]:
# shape of model is 
# input: 26 (A-Z), 1 (Value of letter scaled between 1-99), 5 (None, 2L, 3L, 2W, 3W)
# timestep: 16 (max word length)
# output: score normalised to 0-1000
# type of activation: ReLU
def create_flat_model(timesteps, input_dim):
    alpha = 0.2

    inputs = Input(shape=(timesteps, input_dim))

    x = inputs
    x = Flatten()(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(256)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(64)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(32)(x)
    x = LeakyReLU(alpha)(x)

    x = Dense(16)(x)
    x = LeakyReLU(alpha)(x)
    
    x = Dense(1)(x)
    x = LeakyReLU(alpha)(x)
    
    outputs = x

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [9]:
def extract_training_data(dataframe):
    x_train = []
    y_train = []
    
    bonuses_encoder = ["", "2L", "3L", "2W", "3W"]
    
    for i, row in dataframe.iterrows():
        word = row['word']
        values = eval(row['values'])
        bonuses = eval(row['bonuses'])
        target_score = row['target_score']
        
        x = []
        # convert to 16 time steps
        for char, value, bonus in zip(word, values, bonuses):
            char_vector = [0.0]*26
            char_vector[ord(char)-ord('a')] = 1.0
            
            bonus_vector = [1.0 if bonus_cat == bonus else 0.0 for bonus_cat in bonuses_encoder]
            
            value /= 10
            
            timestep = char_vector + [value,] + bonus_vector
            x.append(timestep)
        
        remaining_columns = 16-len(x)
        for _ in range(remaining_columns):
            x.append([0.0]*(26+1+5))
            
        y = target_score / 100
        #y = target_score
        
        x_train.append(x)
        y_train.append(y)
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    return x_train, y_train

In [78]:
def predict_perfect(X):
    length_mapping = {2: 3, 3: 4, 4: 6, 5: 9, 6: 11, 7: 14}
    # only need word multiplier and value
    bonuses_encoder = ["", "2L", "3L", "2W", "3W"]
    y_pred = []
    for cell in X:
        multiplier = 1
        prod_value = 0
        cell_length = 0
        
        for row in cell:
            bonuses_vect = row[-5:]
            bonuses_index = np.argmax(bonuses_vect, axis=0)
            bonus = bonuses_encoder[bonuses_index]
            if bonus == '2W':
                multiplier *= 2
            elif bonus == '3W':
                multiplier *= 3
            
            value = row[-6]*10
            prod_value += value
        
            if not (row == 0.0).all():
                cell_length += 1
        
        score = multiplier*prod_value + length_mapping.get(cell_length, cell_length*2)
        y_pred.append(score)
    
    return np.array(y_pred)

In [131]:
# generate some random data
def generate_random(total):
    total_length = 16
    input_dim = 26 + 1 + 5

    def onehot_encoder(arr, axis):
        mask = arr == arr.max(axis=axis).reshape((arr.shape[0], arr.shape[1], 1))
        return mask.astype(float)

    x_random = np.random.rand(total, total_length, input_dim)
    x_random[:,:,0:26] = onehot_encoder(x_random[:,:,0:26], axis=2)
    x_random[:,:,-5:] = onehot_encoder(x_random[:,:,-5:], axis=2)

    char_random = np.random.rand(total)
    bonus_random = (np.random.rand(total, total_length, 5) > 0.5)

    #x_random[:,:] = np.where(char_random, x_random[:,:], 0.0)
    for i, char_len in enumerate(char_random):
        length = max(min(int(total_length * char_len), 12), 2)
        x_random[i,length:,:] = 0.0

    x_random[:,:,-5:] = np.where(bonus_random, x_random[:,:,-5:], 0.0)
    
    y_result = predict_perfect(x_random).astype(float) / 100.0
    
    return (x_random, y_result)

In [135]:
dataframe = pd.read_csv("score_prediction/training_data.csv", delim_whitespace=True)
x_train, y_train = extract_training_data(dataframe)
#x_train, y_train = generate_random(10000)

In [14]:
model = create_lstm_model(16, 26+1+5)
#model = create_gru_model(16, 26+1+5)
#model = create_flat_model(16, 26+1+5)

#model = keras.models.load_model("score_prediction/lstm.h5")

In [142]:
def abs_error(y_target, y_pred):
    return np.abs(y_target-y_pred)*100

def rel_error(y_target, y_pred):
    error = np.abs(y_target-y_pred)
    return error / y_target * 100

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-6, decay=1e-8),
    loss="mean_absolute_error",
    metrics=[rel_error, abs_error])

In [143]:
from sklearn.utils import shuffle

x_shuffle, y_shuffle = shuffle(x_train, y_train)

model.fit(
    x=x_shuffle,
    y=y_shuffle,
    validation_split=0.15,
    epochs=10,
    batch_size=32,
    shuffle=True)

Train on 1589 samples, validate on 281 samples
Epoch 1/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.0102 - rel_error: 4.8764 - abs_error: 1.0208 - val_loss: 0.0095 - val_rel_error: 4.4890 - val_abs_error: 0.9486
Epoch 2/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0101 - rel_error: 4.7993 - abs_error: 1.0096 - val_loss: 0.0094 - val_rel_error: 4.4514 - val_abs_error: 0.9391
Epoch 3/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0100 - rel_error: 4.7619 - abs_error: 1.0014 - val_loss: 0.0093 - val_rel_error: 4.4231 - val_abs_error: 0.9341
Epoch 4/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0099 - rel_error: 4.7345 - abs_error: 0.9932 - val_loss: 0.0094 - val_rel_error: 4.4377 - val_abs_error: 0.9393
Epoch 5/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.0099 - rel_error: 4.7061 - abs_error: 0.9885 - val_loss: 0.0093 - val_rel_error: 4.3828 - val_abs_error: 0.9289
Epo

In [146]:
model.save("score_prediction/lstm.h5")
#model.save("score_prediction/gru.h5")
#model.save("score_prediction/dense.h5")

In [144]:
y_predicted = np.round(model.predict(x_train) * 100)
y_predicted = y_predicted.reshape((y_predicted.shape[0],)).astype(int)

results_df = pd.DataFrame(columns=["word", "target_score", "predicted_score", "error", "%error"])
results_df['word'] = dataframe['word']
results_df['target_score'] = dataframe['target_score']
results_df['predicted_score'] = y_predicted

errors = []
per_errors = []
for target_score, predicted_score in zip(results_df['target_score'], y_predicted):
    error = predicted_score-target_score
    per_error = error / target_score * 100
    errors.append(error)
    per_errors.append(f"{per_error:.0f}%")
    
results_df['error'] = errors
results_df['%error'] = per_errors

In [145]:
print(results_df.to_string())

          word  target_score  predicted_score  error %error
0       bestad           143              133    -10    -7%
1        razee            99               93     -6    -6%
2        teaze            99               93     -6    -6%
3          zee            82               58    -24   -29%
4        jatos            81               80     -1    -1%
5      amatols            74               83      9    12%
6       amatol            65               76     11    17%
7        james            65               56     -9   -14%
8      santols            62               71      9    15%
9       razeed            62               63      1     2%
10       adzes            60               59     -1    -2%
11       dazes            60               59     -1    -2%
12      losses            59               61      2     3%
13        kata            58               46    -12   -21%
14        homs            58               50     -8   -14%
15        jism            58            

In [130]:
x_random, y_random = generate_random(100)

def extract_words(X):
    words = []
    for cell in X:
        word = []
        for row in cell:
            char_vect = row[:26]
            if (char_vect == 0).all():
                break
                
            
            char_int = np.argmax(char_vect, axis=0)
            
            char = chr(ord('a')+char_int)
            word.append(char)
        words.append("".join(word))
    
    return np.array(words)
    
words = extract_words(x_random)

target_score = np.round(y_random*100).astype(int)
target_score = target_score.reshape((target_score.shape[0],)).astype(int)

predicted_score = np.round(model.predict(x_random)*100)
predicted_score = predicted_score.reshape((predicted_score.shape[0],)).astype(int)
    
results_df = pd.DataFrame(columns=["word", "target_score", "predicted_score", "error", "%error"])
results_df['word'] = words
results_df['target_score'] = target_score
results_df['predicted_score'] = predicted_score

errors = []
per_errors = []
for target_score, predicted_score in zip(results_df['target_score'], predicted_score):
    error = predicted_score-target_score
    per_error = error / target_score * 100
    errors.append(error)
    
    per_errors.append(f"{per_error:.0f}%")
    
results_df['error'] = errors
results_df['%error'] = per_errors

print(results_df.to_string())

        word  target_score  predicted_score  error %error
0       knxn            66               58     -8   -12%
1   bdtlrohk           332              227   -105   -32%
2   xwoqharx          1046              282   -764   -73%
3        qfd           184               82   -102   -55%
4   ndirkohz           453              326   -127   -28%
5   xjskprxz           134              208     74    55%
6   ayrunbjx           605              411   -194   -32%
7   zxrqyqfz           214              142    -72   -34%
8   omuajekj           333              382     49    15%
9   ueinxquk            65              110     45    69%
10  blodlyye            53               79     26    49%
11        qd            32               17    -15   -47%
12        al           100               52    -48   -48%
13  noocejwu           369              300    -69   -19%
14  oayiiuun           149              162     13     9%
15        md            16               14     -2   -12%
16  saiiuzfp  